## 1) MUTABLE VS IMMUTABLE

In [1]:
def store_lower(_dict, _string):
    """Add a mapping between `_string` and a lowercased version of `_string` to `_dict`

  Args:
    _dict (dict): The dictionary to update.
    _string (str): The string to add.
    """
    orig_string = _string
    _string = _string.lower()
    _dict[orig_string] = _string

d = {}
s = 'Hello'

print(d, 'and', s)

store_lower(d, s)

print('d={}'.format(d), '-> d changed because dictionary is mutable\n'
      , 's={}'.format(s), '-> s didn\'t change because string is immutable')

{} and Hello
d={'Hello': 'hello'} -> d changed because dictionary is mutable
 s=Hello -> s didn't change because string is immutable


### we want to write function to add column to database

####  a. we use mutable variable in the function (When we use this 2nd times, it give use an error)

In [2]:
import pandas as pd

example_df = pd.DataFrame()
example_df['col_mine'] = range(2)

def add_column(values, df=pd.DataFrame()):
    """Add a column of `values` to a DataFrame `df`.
  The column will be named "col_<n>" where "n" is
  the numerical index of the column.

  Args:
    values (iterable): The values of the new column
    df (DataFrame, optional): The DataFrame to update.
      If no DataFrame is passed, one is created by default.

  Returns:
    DataFrame
    """
    df['col_{}'.format(len(df.columns))] = values
    return df

df_function = add_column(range(3))

print(df_function)

add_column(range(3), df_function)

print(df_function)

print(add_column(range(3)))    # when we call 2nd times, it proceeds on same df.

print(add_column(range(2), example_df)) 

# And, it gives us an error because size of column different is from previos df which it proceeds on same df.
print(add_column(range(5))) 

   col_0
0      0
1      1
2      2
   col_0  col_1
0      0      0
1      1      1
2      2      2
   col_0  col_1  col_2
0      0      0      0
1      1      1      1
2      2      2      2
   col_mine  col_1
0         0      0
1         1      1


ValueError: Length of values does not match length of index

#### b. We use immutable variable with df=None

In [3]:
# Use an immutable variable for the default argument 
def better_add_column(values, df=None):
    """Add a column of `values` to a DataFrame `df`.
  The column will be named "col_<n>" where "n" is
  the numerical index of the column.

  Args:
    values (iterable): The values of the new column
    df (DataFrame, optional): The DataFrame to update.
      If no DataFrame is passed, one is created by default.

  Returns:
    DataFrame
    """
  # Update the function to create a default DataFrame
    if df is None:
        df = pd.DataFrame()
    df['col_{}'.format(len(df.columns))] = values
    return df

df_function = better_add_column(range(3))

print(df_function)

better_add_column(range(3), df_function)

print(df_function)

print(better_add_column(range(3)))  # when we write special df, it produces a new df.

print(better_add_column(range(2)))

   col_0
0      0
1      1
2      2
   col_0  col_1
0      0      0
1      1      1
2      2      2
   col_0
0      0
1      1
2      2
   col_0
0      0
1      1


#### When you need to set a mutable variable as a default argument, always use None and then set the value in the body of the function. This prevents unexpected behavior like adding multiple columns if you call the function more than once.

## 2) Context Manager
#### timer() context manager to figure out which some functions is running too slow.

In [4]:
import contextlib
import time

# Add a decorator that will make timer() a context manager
@contextlib.contextmanager
def timer():
    """Time the execution of a context block.

    Yields:
        None
    """
    start = time.time()
    # Send control back to the context block
    yield
    end = time.time()
    print('Elapsed: {:.2f}s'.format(end - start))

with timer():
    print('This should take approximately 0.25 seconds')
    time.sleep(0.25)

This should take approximately 0.25 seconds
Elapsed: 0.28s


In [5]:
@contextlib.contextmanager
def my_context():
    print('hello')
    yield 42
    print('good bye')

with my_context() as my_con: 
    print('foo is {}'.format(my_con))

hello
foo is 42
good bye


In [6]:
import os
@contextlib.contextmanager
def in_dir(directory):
    """Change current working directory to `directory`,
  allow the user to run some code, and change back.

  Args:
    directory (str): The path to a directory to work in.
    """
    current_dir = os.getcwd()
    os.chdir(directory)

    # Add code that lets you handle errors
    try:
        yield
    # Ensure the directory is reset,
    # whether there was an error or not
    finally:
        os.chdir(current_dir)

## 3) Decorators
### To control whether functons have docstring or not  

In [7]:
def has_docstring(func):
    """Check to see if the function 
    `func` has a docstring.

    Args:
        func (callable): A function.
    Returns:
    bool
    """
    return func.__doc__ is not None

# Call has_docstring() on the log_product() function
ok = has_docstring(better_add_column)

if not ok:
    print("better_add_column() doesn't have a docstring!")
else:
    print("better_add_column() looks ok")

better_add_column() looks ok


### Returning functions for a math game

In [8]:
def create_math_function(func_name):
    if func_name == 'add':
        def add(a, b):
            return a + b
        return add
    elif func_name == 'subtract':
        # Define the subtract() function
        def subtract(a, b):
            return a - b
        return subtract
    else:
        print("I don't know that one")
    
add = create_math_function('add')
print('5 + 2 = {}'.format(add(5, 2)))

subtract = create_math_function('subtract')
print('5 - 2 = {}'.format(subtract(5, 2)))

5 + 2 = 7
5 - 2 = 3


## 4) Global / Nonlocal / Local 

In [9]:
x = 50

def one():
    x = 10

def two():
    global x
    x = 30

def three():
    x = 100
    print(x)
    
    def four():
        
        #nonlocal x
        print(x)
    four()

for func in [one, two, three]:
    func()

    print(func.__name__, x)

one 50
two 30
100
100
three 30


In [10]:
def read_files():
    file_contents = None
  
    def save_contents(filename):
        # Add a keyword that lets us modify file_contents
        nonlocal file_contents
        if file_contents is None:
            file_contents = []
        with open(filename, encoding='utf8') as fin:
            file_contents.append(fin.read())
      
    for filename in ['Datasets/emre.txt', 'Datasets/sumeyye.txt']:
        save_contents(filename)
    
    return file_contents

print(''.join(read_files()))

I love you sumeyye


In [11]:
import random

def wait_until_done():
    def check_is_done():
        # Add a keyword so that wait_until_done() 
        # doesn't run forever
        global done
        if random.random() < 0.1:
            done = True
      
    while not done:
        check_is_done()

done = False
wait_until_done()

print('Work done? {}'.format(done))

Work done? True


## 5) Closure

In [12]:
def return_a_func(arg1, arg2):
    def new_func():
        print('arg1 was {}'.format(arg1))
        print('arg2 was {}'.format(arg2))
    return new_func
    
my_func = return_a_func(2, 17)

print(my_func.__closure__ is not None)
print(len(my_func.__closure__) == 2)

# Get the values of the variables in the closure
closure_values = [
  my_func.__closure__[i].cell_contents for i in range(2)
]
print(closure_values == [2, 17])

True
True
True


In [21]:
def my_special_function():
    print('You are running my_special_function()')  

def get_new_func(func):
    def call_func():
        func()
    return call_func

new_func = get_new_func(my_special_function)

# Redefine my_special_function() to just print "hello"
def my_special_function():
    print("hello")

new_func()

# Overwrite `my_special_function` with the new function
new2_func = get_new_func(my_special_function)

new2_func()

# Delete my_special_function()
del(my_special_function)

new_func()

You are running my_special_function()
hello
You are running my_special_function()


#### You could run into memory issues if you wound up adding a very large array or object to the closure, and has resolved to keep her eye out for that sort of problem.

## 6) Decorator

In [32]:
def my_function(a, b, c):
    print(a + b + c)

def print_args(func):
    
    def wrapper(a, b, c):
        
        print('function was called with a={}, b={}, c={}'.format(a, b, c))
        
        return func(a, b, c)
    
    return wrapper

# Decorate my_function() with the print_args() decorator
my_function = print_args(my_function)

my_function(1, 2, 3)

function was called with a=1, b=2, c=3
6


In [34]:
# Decorate your_function() with the print_args() decorator
@print_args
def your_function(a, b, c):
    print(a * b * c)

your_function(4, 5, 6)

function was called with a=4, b=5, c=6
120


In [44]:
def print_before_and_after(func):
    def wrapper(*args):
        print('Before {}'.format(func.__name__))
        # Call the function being decorated with *args
        print('function was called with ', *args)
        func(*args)
        print('After {}'.format(func.__name__))
    # Return the nested function
    return wrapper

@print_before_and_after
def multiply(a, b):
    print(a * b)

multiply(5, 10)

print('\n')

@print_before_and_after
def add(a, b, c, d):
    print(a+b+c+d)

add(3,4,5,6)

Before multiply
function was called with  5 10
50
After multiply


Before add
function was called with  3 4 5 6
18
After add


In [50]:
def print_return_type(func):
    # Define wrapper(), the decorated function
    def wrapper(*args, **kwargs):
        # Call the function being decorated
        result = func(*args, **kwargs)
        print('{}() returned type {}'.format(
          func.__name__, type(result)
        ))
        return result
    # Return the decorated function
    return wrapper
  
@print_return_type
def foo(value):
    return value
  
print(foo(42))
print('*********')
print(foo([1, 2, 3]))
print('*********')
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
*********
foo() returned type <class 'list'>
[1, 2, 3]
*********
foo() returned type <class 'dict'>
{'a': 42}


####  You're working on a new web app, and you are curious about how many times each of the functions in it gets called. So you decide to write a decorator that adds a counter to each function that you decorate. You could use this information in the future to determine whether there are sections of code that you could remove because they are no longer being used by the app.

In [56]:
def counter(func):
    def wrapper(*args, **kwargs):
        print('Enter the wrapper fuction')
        wrapper.count += 1
        # Call the function being decorated and return the result
        return func
    print('Enter the counter fuction')
    wrapper.count = 0
    # Return the new decorated function
    return wrapper

# Decorate foo() with the counter() decorator
@counter
def foo():
    print('calling foo()')

foo()
foo()
foo()

print('foo() was called {} times.'.format(foo.count))

Enter the counter fuction
Enter the wrapper fuction
Enter the wrapper fuction
Enter the wrapper fuction
foo() was called 3 times.


## 7) Decorator and Metadata

In [60]:
def add_hello(func):
    def wrapper(*args, **kwargs):
        """Print 'hello' and then call the decorated function."""
        print('Hello')
        return func(*args, **kwargs)
    return wrapper

# Decorate print_sum() with the add_hello() decorator
@add_hello
def print_sum(a, b):
    """Adds two numbers and prints the sum"""
    print(a + b)

print_sum(10, 20)
print('************')
print(print_sum.__doc__)

Hello
30
************
Print 'hello' and then call the decorated function.


it is written wrapper's doc because sum have a decorator.

#### import 'from functools import wraps' and add '@wraps(func)' inside decorator. So, you can write original funtion sum's doc.

In [63]:
# Import the function you need to fix the problem
from functools import wraps

def add_hello(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """Print 'hello' and then call the decorated function."""
        print('Hello')
        return func(*args, **kwargs)
    return wrapper

# Decorate print_sum() with the add_hello() decorator
@add_hello
def print_sum(a, b):
    """Adds two numbers and prints the sum"""
    print(a + b)

print_sum(10, 20)
print('************')
print(print_sum.__doc__)

Hello
30
************
Adds two numbers and prints the sum


#### Your boss wrote a decorator called add_hello() that they think is amazing, and they are insisting you use it on your function. However, you've noticed that when you use it to decorate your functions, it makes them run much slower. You need to convince your boss that the decorator is adding too much processing time to your function. To do this, you are going to measure how long the decorated function takes to run and compare it to how long the undecorated function would have taken to run. 

In [67]:
@add_hello
def duplicate(my_list):
    """Return a new list that repeats the input twice"""
    return my_list + my_list

t_start = time.time()
duplicated_list = duplicate(list(range(50)))
t_end = time.time()
decorated_time = t_end - t_start

t_start = time.time()
# Call the original function instead of the decorated one
duplicated_list = duplicate.__wrapped__(list(range(50)))
t_end = time.time()
undecorated_time = t_end - t_start

print('Decorated time: {:.5f}s'.format(decorated_time))
print('Undecorated time: {:.5f}s'.format(undecorated_time))

Hello
Decorated time: 0.01563s
Undecorated time: 0.00000s


#### You are not right, because the times are almost same. In my opinion, you have to listen to your boss and use the decorator add_hello

### HTML Generator
You are writing a script that generates HTML for a webpage on the fly. So far, you have written two decorators that will add bold or italics tags to any function that returns a string. You notice, however, that these two decorators look very similar. Instead of writing a bunch of other similar looking decorators, you want to create one decorator, html(), that can take any pair of opening and closing tags.

In [73]:
def bold(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        msg = func(*args, **kwargs)
        return '<b>{}</b>'.format(msg)
    return wrapper

In [74]:
def italics(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        msg = func(*args, **kwargs)
        return '<i>{}</i>'.format(msg)
    return wrapper

In [68]:
def html(open_tag, close_tag):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            msg = func(*args, **kwargs)
            return '{}{}{}'.format(open_tag, msg, close_tag)
        # Return the decorated function
        return wrapper
    # Return the decorator
    return decorator

In [75]:
# Make hello() return bolded text
@bold
def hello(name):
    return 'Hello {}!'.format(name)

print(hello('Alice'))

<b>Hello Alice!</b>


In [69]:
# Make hello() return bolded text
@html('<b>', '</b>')
def hello(name):
    return 'Hello {}!'.format(name)

print(hello('Alice'))

<b>Hello Alice!</b>


In [76]:
# Make goodbye() return italicized text
@italics
def goodbye(name):
    return 'Goodbye {}.'.format(name)
  
print(goodbye('Alice'))

<i>Goodbye Alice.</i>


In [77]:
# Make goodbye() return italicized text
@html('<i>', '</i>')
def goodbye(name):
    return 'Goodbye {}.'.format(name)
  
print(goodbye('Alice'))

<i>Goodbye Alice.</i>


In [72]:
# Wrap the result of hello_goodbye() in <div> and </div>
@html('<div>', '</div>')
def hello_goodbye(name):
    return '\n{}\n{}\n'.format(hello(name), goodbye(name))
  
print(hello_goodbye('Alice'))

<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>


## Tag your functions
Tagging something means that you have given that thing one or more strings that act as labels. For instance, we often tag emails or photos so that we can search for them later. You've decided to write a decorator that will let you tag your functions with an arbitrary list of tags.

In [78]:
def tag(*tags):
  # Define a new decorator, named "decorator", to return
  def decorator(func):
    # Ensure the decorated function keeps its metadata
    @wraps(func)
    def wrapper(*args, **kwargs):
      # Call the function being decorated and return the result
      return func(*args, **kwargs)
    wrapper.tags = tags
    return wrapper
  # Return the new decorator
  return decorator

@tag('test', 'this is a tag')
def foo():
  pass

print(foo.tags)

('test', 'this is a tag')


## Check the return type
Python's flexibility around data types is usually cited as one of the benefits of the language. It can occasionally cause problems though if incorrect data types go unnoticed. You've decided that in order to make sure your code is doing exactly what you want it to do, you will explicitly check the return types of all of your functions and make sure they are what you expect them to be. To do that, you are going to create a decorator that checks that the return type of the decorated function is correct.

In [79]:
def returns_dict(func):
  # Complete the returns_dict() decorator
  def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)
    assert(type(result) == dict)
    return result
  return wrapper

@returns_dict
def foo(value):
  return value

try:
  print(foo([1,2,3]))
except AssertionError:
  print('foo() did not return a dict!')

foo() did not return a dict!


In [80]:
def returns(return_type):
  # Write a decorator that raises an AssertionError if the
  # decorated function returns a value that is not return_type
  def decorator(func):
    def wrapper(*args, **kwargs):
      result = func(*args, **kwargs)
      assert(type(result) == return_type)
      return result
    return wrapper
  return decorator
  
@returns(dict)
def foo(value):
  return value

try:
  print(foo([1,2,3]))
except AssertionError:
  print('foo() did not return a dict!')

foo() did not return a dict!
